In [2]:
pip install gensim

     |████████████████████████████████| 24.2 MB 2.2 MB/s eta 0:00:01     |██████████████▉                 | 11.2 MB 296 kB/s eta 0:00:44
     |████████████████████████████████| 106 kB 203 kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.0.0-py3-none-any.whl size=108253 sha256=8dd14c2240e207d02927a78dbd24aa5179884d0063a4dbf90a265aa21badb90b
  Stored in directory: /Users/mmmujin/Library/Caches/pip/wheels/c3/5c/27/33e05bd883bc60454a549d97103c68934d0ea877ab5fdb69c2
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from pprint import pprint
import numpy as np
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mmmujin/anaconda3/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
data = pd.read_csv('covid_cleandf.csv', error_bad_lines=False);
data_text = data[['text']]
data_text['index'] = data_text.index
documents = data_text
print(len(documents))
print(documents[:5])

12830
                                                text  index
0  WKND Inspired Living KT Home Videos Interactiv...      0
1  Virus transmitted to people from wild animals ...      1
2  The humanitarian sector has a trust problem Th...      2
3  Alabama Mobile Researchers from the University...      3
4  WKND Inspired Living KT Home Videos Interactiv...      4


<ipython-input-5-9c2ff2a05b8f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_text['index'] = data_text.index


In [6]:
stemmer = SnowballStemmer('english')
 
#Extract stemming, part of speech restoration
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
 
#Filter stop words and delete words less than 3 in length.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [17]:
#This is just a test of preprocess
doc_sample = documents[documents['index'] == 100].values[0][0]
 
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['UPS', 'To', 'Airlift', 'More', 'Than', '2', 'Million', 'Masks', 'And', 'Protective', 'Gear', 'To', 'China', 'ATLANTA', 'Jan', '31', '2020', 'GLOBE', 'NEWSWIRE', 'The', 'UPS', 'Foundation', 'which', 'leads', 'the', 'global', 'citizenship', 'programs', 'for', 'UPS', 'NYSE', 'UPS', 'announced', 'today', 'it', 'is', 'providing', 'free', 'air', 'transportation', 'of', 'more', 'than', '2', 'million', 'respirator', 'masks', 'and', '11000', 'protective', 'coveralls', 'to', 'China', 'to', 'help', 'combat', 'the', 'spread', 'of', 'the', 'coronavirus', 'in', 'Wuhan', 'China', 'Working', 'with', 'MAP', 'International', 'and', 'MedShare', 'two', 'Georgiabased', 'nonprofit', 'global', 'health', 'organizations', 'the', 'UPS', 'operated', 'flight', 'is', 'being', 'coordinated', 'through', 'the', 'Red', 'Cross', 'Society', 'of', 'China', 'and', 'the', 'recipient', 'organization', 'Hubei', 'Provincial', 'Center', 'for', 'Disease', 'Control', 'and', 'Prevention', 'in', 'collaboratio

In [7]:
processed_docs = documents['text'].map(preprocess)
processed_docs[:10]

0    [wknd, inspir, live, home, video, interact, po...
1    [virus, transmit, peopl, wild, anim, spread, b...
2    [humanitarian, sector, trust, problem, existen...
3    [alabama, mobil, research, univers, south, ala...
4    [wknd, inspir, live, home, video, interact, po...
5    [wknd, inspir, live, home, video, interact, po...
6    [wknd, inspir, live, home, video, interact, po...
7    [wknd, inspir, live, home, video, interact, po...
8    [wknd, inspir, live, home, video, interact, po...
9    [wknd, inspir, live, home, video, interact, po...
Name: text, dtype: object

In [8]:
#Bag of words
dictionary = gensim.corpora.Dictionary(processed_docs)
for i in range(10):
    print(i,dictionary[i])
print('number of total words:',len(dictionary))

0 abdulla
1 abroad
2 accid
3 accomplish
4 accord
5 activ
6 add
7 adnoc
8 adult
9 africa
number of total words: 111036


In [9]:
#The number is the number of documents which the first 10 words appear, not the word frequency.
for i in range(10):
    print(i,dictionary.dfs[i])

0 22
1 1257
2 253
3 187
4 6076
5 2596
6 5156
7 57
8 604
9 2164


In [10]:
#Filter out the words whose frequency is less than 5 times
#Filter out more than 50% of the total word frequency
#only the first 100000 most frequent words are retained
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
len(dictionary)
print('number of total words:',len(dictionary))

number of total words: 12588


In [11]:
#Count all words in each document
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(processed_docs[0])
print(bow_corpus[0])
#0 in (0,1) represents the index number of the word "wknd" in the dictionary
#1 represents the number of times the word "wknd" appears in the current document

['wknd', 'inspir', 'live', 'home', 'video', 'interact', 'podcast', 'emir', 'mar', 'mission', 'univers', 'independ', 'educ', 'enrol', 'cookbook', 'remot', 'workforc', 'summit', 'wknd', 'young', 'time', 'inspir', 'live', 'event', 'eventspast', 'event', 'dubai', 'care', 'volunt', 'bring', 'hope', 'children', 'remot', 'villag', 'seneg', 'divers', 'group', 'repres', 'nation', 'profession', 'background', 'rigor', 'select', 'dubai', 'care', 'thirteen', 'uaebas', 'volunt', 'success', 'complet', 'volunt', 'mission', 'villag', 'seneg', 'particip', 'construct', 'school', 'divers', 'group', 'repres', 'nation', 'profession', 'background', 'rigor', 'select', 'dubai', 'care', 'moham', 'rashid', 'maktoum', 'global', 'initi', 'form', 'volunt', 'forc', 'second', 'edit', 'popular', 'volunt', 'global', 'initi', 'novemb', 'volunt', 'join', 'local', 'communiti', 'foundat', 'communitybas', 'school', 'benefit', 'children', 'femal', 'enrol', 'illiter', 'adult', 'direct', 'adult', 'communiti', 'indirect', 'scho

In [12]:
bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

Word 0 ("abdulla") appears 1 time.
Word 1 ("abroad") appears 1 time.
Word 2 ("accid") appears 2 time.
Word 3 ("accomplish") appears 1 time.
Word 4 ("accord") appears 1 time.
Word 5 ("activ") appears 1 time.
Word 6 ("add") appears 1 time.
Word 7 ("adnoc") appears 1 time.
Word 8 ("adult") appears 2 time.
Word 9 ("africa") appears 1 time.
Word 10 ("agreement") appears 1 time.
Word 11 ("ahm") appears 1 time.
Word 12 ("airlin") appears 1 time.
Word 13 ("announc") appears 1 time.
Word 14 ("anti") appears 1 time.
Word 15 ("appli") appears 1 time.
Word 16 ("appreci") appears 1 time.
Word 17 ("approv") appears 2 time.
Word 18 ("arab") appears 1 time.
Word 19 ("arriv") appears 1 time.
Word 20 ("ask") appears 1 time.
Word 21 ("august") appears 10 time.
Word 22 ("avail") appears 1 time.
Word 23 ("background") appears 2 time.
Word 24 ("bag") appears 1 time.
Word 25 ("belief") appears 1 time.
Word 26 ("benefit") appears 1 time.
Word 27 ("best") appears 1 time.
Word 28 ("bond") appears 1 time.
Word 2

In [18]:
#Train LDA model on bag of words corpus
lda_model = gensim.models.LdaMulticore(bow_corpus, id2word=dictionary, passes=2, workers=2)

In [19]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.022*"lebanon" + 0.015*"explos" + 0.014*"diab" + 0.013*"blast" + 0.012*"beirut" + 0.012*"minist" + 0.012*"port" + 0.007*"lebanes" + 0.006*"emerg" + 0.005*"support"
Topic: 1 
Words: 0.026*"beirut" + 0.024*"lebanon" + 0.021*"explos" + 0.015*"lebanes" + 0.014*"blast" + 0.010*"port" + 0.007*"saudi" + 0.006*"minist" + 0.006*"damag" + 0.006*"injur"
Topic: 2 
Words: 0.033*"yemen" + 0.014*"yemeni" + 0.012*"houthi" + 0.010*"fertilis" + 0.007*"hospit" + 0.006*"aden" + 0.006*"citi" + 0.006*"infect" + 0.006*"sanaa" + 0.005*"nasser"
Topic: 3 
Words: 0.012*"explos" + 0.008*"lebanon" + 0.006*"know" + 0.006*"blast" + 0.006*"like" + 0.005*"minist" + 0.005*"go" + 0.005*"nang" + 0.005*"port" + 0.005*"august"
Topic: 4 
Words: 0.039*"yemen" + 0.014*"houthi" + 0.009*"yemeni" + 0.008*"peac" + 0.008*"coalit" + 0.008*"forc" + 0.007*"rebel" + 0.007*"ceasefir" + 0.007*"conflict" + 0.007*"afghanistan"
Topic: 5 
Words: 0.011*"food" + 0.007*"sheikha" + 0.006*"refuge" + 0.005*"provid" + 0.005*"supp

In [21]:
#Randomly select a document to view the classification of the topic in the LDA bag of words model
docId=100
print(documents['text'][docId])
print(processed_docs[docId])
print('-------------------------------------')
for index, score in sorted(lda_model[bow_corpus[docId]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic{}: {}".format(score,index, lda_model.print_topic(index, 10)))

UPS To Airlift More Than 2 Million Masks And Protective Gear To China ATLANTA Jan 31 2020 GLOBE NEWSWIRE The UPS Foundation which leads the global citizenship programs for UPS NYSE UPS announced today it is providing free air transportation of more than 2 million respirator masks and 11000 protective coveralls to China to help combat the spread of the coronavirus in Wuhan China Working with MAP International and MedShare two Georgiabased nonprofit global health organizations the UPS operated flight is being coordinated through the Red Cross Society of China and the recipient organization Hubei Provincial Center for Disease Control and Prevention in collaboration with Project HOPE The world needs strong publicprivate partnerships to help contain the spread of this deadly virus and The UPS Foundation is expanding its humanitarian relief network to support our partners in providing supply chain expertise and air transport said Eduardo Martinez president of The UPS Foundation and UPS chief